<a href="https://colab.research.google.com/github/DataParadox/iTrustBD/blob/main/Codes/iTrustBD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Clone Project from GitHub and Connect to Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd '/content/drive/My Drive/PG/CSE6413 Network Science/Project Works/'

/content/drive/My Drive/PG/CSE6413 Network Science/Project Works


In [ ]:
!git clone 'https://github.com/DataParadox/iTrustBD.git'

In [3]:
%cd iTrustBD

/content/drive/My Drive/PG/CSE6413 Network Science/Project Works/iTrustBD


In [4]:
!git pull

remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 4 (delta 1), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (4/4), done.
From https://github.com/DataParadox/iTrustBD
   d7522f9..72b4f91  main       -> origin/main
Updating d7522f9..72b4f91
Fast-forward
 Codes/iTrustBD.ipynb | 859 +++++++++++++++++++++++++++++++++++++++++++++------
 1 file changed, 761 insertions(+), 98 deletions(-)


# Set Datasets

In [5]:
def setDatasets(dataset):           # dataset -> Name of the dataset, i.e. 'soc-sign-bitcoinalpha-dataset' 
  import pandas as pd
  
  nodeTable = pd.read_csv('./BTC-Alpha/'+dataset+'-nodes-table.csv')
  edgeTable = pd.read_csv('./BTC-Alpha/'+dataset+'-edge-table.csv')

  return nodeTable, edgeTable


dataset = 'soc-sign-bitcoinalpha-dataset'
nodeTable, edgeTable = setDatasets(dataset)

In [6]:
nodeTable['behavior'] = nodeTable['Class']
nodeTable

,Id,Class,timeset,indegree,outdegree,Degree,weighted indegree,weighted outdegree,Weighted Degree,Eccentricity,...,Authority,Hub,pageranks,componentnumber,strongcompnum,modularity_class,stat_inf_class,clustering,eigencentrality,behavior
0,7604,-1,NaN,73,21,94,-628,110,-518,6,...,0.080527,0.025972,NaN,0,506,2,39,0.097339,0.388011,-1
1,5342,-1,NaN,39,80,119,0,-249,-249,6,...,0.048157,0.049823,NaN,0,506,13,32,0.040049,0.241042,-1
2,7602,-1,NaN,17,13,30,-150,10,-140,7,...,0.027852,0.021469,NaN,0,506,2,19,0.338768,0.142756,-1
3,7594,-1,NaN,8,6,14,-61,-60,-121,7,...,0.008680,0.007923,NaN,0,506,13,93,0.138889,0.047984,-1
4,708,-1,NaN,7,24,31,12,-128,-116,7,...,0.007487,0.011672,NaN,0,506,13,44,0.035000,0.035902,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3778,6,1,NaN,139,148,287,388,252,640,5,...,0.122799,0.122040,NaN,0,506,13,77,0.052608,0.633059,1
3779,4,1,NaN,201,215,416,588,476,1064,5,...,0.104260,0.110217,NaN,0,506,2,18,0.026903,0.568198,1
3780,3,1,NaN,251,243,494,610,493,1103,6,...,0.177256,0.161742,NaN,0,506,13,38,0.035077,0.900561,1
3781,2,1,NaN,205,195,400,735,397,1132,5,...,0.173652,0.160447,NaN,0,506,8,13,0.048873,0.869056,1


In [7]:
edgeTableTargetSource = edgeTable[['Target', 'Source']].astype(str)
neighborTable = edgeTableTargetSource.groupby(['Target'], as_index=False).agg(lambda col: ', '.join(col))
neighborTable.columns = neighborTable.columns.str.replace('Source', 'Neighbors')
neighborTable

,Target,Neighbors
0,1,"7188, 430, 3134, 3026, 3010, 804, 160, 95, 377..."
1,10,"1, 2, 3, 4, 8, 37, 177, 623, 130, 334, 62, 161..."
2,100,"3, 5, 6, 7, 11, 12, 14, 16, 21, 26, 44, 50, 52..."
3,1000,"509, 772"
4,1001,"39, 132, 340, 368, 403, 573, 580"
...,...,...
3749,995,"166, 504, 608"
3750,996,"168, 353, 1615"
3751,997,"129, 848"
3752,998,"129, 473, 775"


# Generate Random Probability

In [8]:
def randProbability(min, max):
  # generate random floating point values
  import random
  return min + (random.random() * (max - min))

# Behavior Spread Based on Mutation Factor

In [9]:
def behaviorSpreadMutate():
  import random
  meu_m = randProbability(0, 0.01)                                     # Random mutation factor generation in the range of (0, 0.01)
  for trader in nodeTable.index:
    if nodeTable.iloc[trader].Class == 0:
      if randProbability(0, 1) >= (1-meu_m):                           # If cross the threshold of meu_m
        nodeTable.iloc[trader].behavior = random.choice([-1, 0, 1])   # Change behavior randomly

# Behavior Spread Based on Neighbors

In [46]:
def behaviorSpreadNeighbors():
  import random
  gamma = randProbability(0, 0.05)       # Random transmission rate generation in the range of (0, 0.05)
  for trader in nodeTable.index:
    if nodeTable.iloc[trader].Class == 0:
      if randProbability(0, 1) >= (1-gamma):
        try:
          random_neighbor = random.choice(neighborTable.loc[neighborTable['Target'].astype(int) == nodeTable.iloc[trader].Id.astype(int)]['Neighbors'].to_string(index=False).split(","))
          nodeTable.iloc[trader].behavior = int(nodeTable.loc[nodeTable['Id'].astype(int) == int(random_neighbor)].behavior.to_string(index=False))
        except:
          pass

# Setting of Parameters

In [48]:
def setParameters():
  beta_T = randProbability(0, 1)
  beta_S = beta_T

  meu_T = randProbability(0, beta_T)
  meu_S = meu_T

  return beta_T, beta_S, meu_T, meu_S

# Calculating Propotion of Neighbors

In [ ]:
def neighborProportion(nodeId):
  print('Code goes here')             # Value of sigma_C_i
neighborProportion()

Code goes here


# Susceptible-Infected-Susceptible (SIS) Model

In [ ]:
def modelSIS():
  print('Codes goes here')              # Codes goes here
modelSIS()

Codes goes here


# Voter Model

In [ ]:
def modelVoter():
  print('Codes goes here')              # Codes goes here
modelVoter()

Codes goes here


# Bilingual Model

In [ ]:
def modelBilingual():
  print('Codes goes here')              # Codes goes here
modelBilingual()

Codes goes here


# Behavior Spread

In [ ]:
def behaviorSpread():
  epoch = 0
  total_epoch = 10000
  # Set all nodes ignorant
  # Set one random node aware
  while epoch < total_epoch:
    epoch = epoch + 1
    behaviorSpreadNeighbors()
    behaviorSpreadMutate()
    #for i in neutral_traders:
      # Select a random neighbor j with probability proportional to in_degree.
      #if i neutral & j not neutral:
        # i can assume the state of j under the rules of the model
      #elif i not neutral:
          # set i neutral under the rules of the model